In [5]:
import pdfplumber
import re

In [6]:
pdf_path = r"..\data\ricette_to_elaborate\Le-ricette-di-stagioni-nel-piatto-ricettario-ebook.pdf"

In [33]:
import pdfplumber

def estrai_colonne_pdf(pdf_path):
    """Estrae il testo da un PDF dove il contenuto è diviso in due colonne (sinistra e destra) in base alla parola 'Procedimenti'."""
    testo_completo = {}
    tipo_regex = {
        "dolce": r"\bdolci?\b",  # Cerca "dolce" o "dolci"
        "antipasto": r"\b(antipasto|antipasti)\b",  # Cerca "antipasto" o "antipasti"
        "primo": r"\b(primo piatto|primi piatti)\b",  # Cerca "primo piatto" o "primi piatti"
        "secondo": r"\b(secondo piatto|secondi piatti|secondo)\b"  # Cerca "secondo piatto", "secondi piatti", "secondo"
    }
    with pdfplumber.open(pdf_path) as pdf:
        for i, pagina in enumerate(pdf.pages[:15]):  # Escludiamo le prime 9 pagine
            # Ottieni tutte le parole della pagina insieme alle loro posizioni (x0, y0, x1, y1)
            words = pagina.extract_words()

            # Trova la posizione della parola "Procedimenti"
            procedimenti_pos_y = None
            procedimenti_pos_x = None
            for word in words:
                #print(word)
                if "procedimento" in word['text'].lower():
                    procedimenti_pos_x = word['x0']  # Salviamo la posizione della parola "Procedimenti"
                    procedimenti_pos_y = word['bottom']
                    break
                
            
            if procedimenti_pos_x is not None and procedimenti_pos_y is not None:
                # Dividiamo il testo in ingredienti e procedimento
                ingredienti = []
                procedimento = []
                nome = ""
                for word in words:
                    # Se la parola è a sinistra di "Procedimenti", è un ingrediente
                    if word['x0'] < procedimenti_pos_x  and word['bottom'] > procedimenti_pos_y:
                            ingredienti.append(word)
                    # Se la parola è a destra di "Procedimenti", è il procedimento
                    elif word['x0'] >= procedimenti_pos_x and word['bottom'] > procedimenti_pos_y:
                        procedimento.append(word)
                    elif word['bottom'] < procedimenti_pos_y:
                        for tipo_key, regex in tipo_regex.items():
                            if re.search(regex, word['text'], re.IGNORECASE):
                                tipo = tipo_key
                        if word["bottom"] < 100:
                            nome += " " + word["text"]

                                  
                 
                # Processamento degli ingredienti
                rows_ingredienti = ""
                ingredienti_list = []
                previous_word = None
                for word in ingredienti:
                    if previous_word is not None:
                        if word["bottom"] == previous_word["bottom"]:  # Stessa riga
                            rows_ingredienti += " " + word["text"]  # Aggiungi spazio tra le parole
                        else:
                            if rows_ingredienti.strip() and "per" not in rows_ingredienti.lower():  # Se la riga non è vuota
                                ingredienti_list.append(rows_ingredienti.strip())
                            rows_ingredienti = word["text"]  # Inizia una nuova riga
                    else:
                        rows_ingredienti += word["text"]
                    previous_word = word
                
                # Aggiungi l'ultimo ingrediente (se c'è)
                if rows_ingredienti.strip():
                    ingredienti_list.append(rows_ingredienti.strip())

                # Processamento del procedimento
                rows_procedimenti = ""
                procedimenti_list = []
                previous_word = None
                for word in procedimento:
                    if previous_word is not None:
                        if word["bottom"] == previous_word["bottom"]:  # Stessa riga
                            rows_procedimenti += " " + word["text"]  # Aggiungi spazio tra le parole
                        else:
                            procedimenti_list.append(rows_procedimenti.strip())
                            rows_procedimenti = word["text"]  # Inizia una nuova riga
                    else:
                        rows_procedimenti += word["text"]
                    previous_word = word
                
                # Aggiungi l'ultimo passo del procedimento (se c'è)
                if rows_procedimenti.strip():
                    procedimenti_list.append(rows_procedimenti.strip())

                # Combina le parole in una stringa per ingredienti e procedimento
                testo_completo[i] = {
                    "nome": nome.strip(),
                    "tipo": tipo,
                    "ingredienti":ingredienti_list,
                    "procedimento": "\n".join(procedimenti_list)
                }
    
    return testo_completo
result = estrai_colonne_pdf(pdf_path)

In [38]:
output_file = 'recipes.json'
import json 
# Write the dictionary to a JSON file
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)